In [260]:
%run utils.ipynb
from bs4 import BeautifulSoup
import requests
import sys
import time
import pandas as pd
import urllib.robotparser as urobot
from tqdm import tqdm_notebook as tqdm
import validators
import os

header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/605.1.15'}

urls = [
        "https://www.magazineluiza.com.br",
        "https://www.saraiva.com.br",
        "https://www.amazon.com.br",
        "https://www.ricardoeletro.com.br",
        "https://www.kabum.com.br",
        "https://www.ibyte.com.br",
        "https://www.cissamagazine.com.br",
        "https://www.promobit.com.br",
        "https://www.havan.com.br",
        "https://www.avenida.com.br"]

#urls = ["https://www.avenida.com.br"]

In [261]:
#Class definition of the bfs_crawler
class bfs_crawler:
    def __init__(self, url):
        self.url = url                
        self.robotParser = getRobot(url)      #robot parser to check if a link is valid or not
        self.links_list = []                  #list of allowed links
        self.invalid_links = []                    #list of invalid links
        self.file_name  = set_file_name(url)  #name of the files generate by the crawler 
        
#     def __init__(self):
#         self.url = None
#         self.robotParser = None
#         self.links_list = []
    
    def get_links(self):
        actual_link = self.url
        link_count = 0
        index = 0
        number_of_links = 1000
        visited_links = [self.url]
        
        while (len(self.links_list) < number_of_links):
            try:
                req = requests.get(actual_link, headers=header)
                if(req.status_code == 200):    
                    soup = BeautifulSoup(req.text)
                    pageLinks = soup.findAll("a", href=True)
                    
                    for a in pageLinks:
                        #formata o link na tentativa de obter um link válido
                        link = format_link(self.url, a["href"])  
                        #verifica se o link é valido
                        if(not validators.url(link)):
                            self.invalid_links.append(link)
                            
                        #adiciona na lista de links do crawler caso seja um link não visitado, válido e que o robots.txt permita
                        elif(self.robotParser.can_fetch("*", link) and link not in visited_links):
                            #print("{} - {}".format(link_count, link))
                            self.links_list.append(link)
                            visited_links.append(link)
                            link_count += 1
                        
                    
            except Exception:
                self.invalid_links.append(link)
            finally: 
                if(index >= len(self.links_list) or len(self.links_list) >= number_of_links):
                    print("END")
                    return
                actual_link = self.links_list[index]
                print("Link Atual: {}".format(actual_link))
                index += 1
                time.sleep(1)
                
    #saves the links as csv 
    def save_as_csv(self):
        folders = ["links", "invalid_links"]
        for folder in folders:
            if not os.path.exists(folder):
                os.makedirs(folder)
                
        df = pd.DataFrame(self.links_list, columns=["links"])
        df.to_csv(('links/' + self.file_name + '.csv'),header=True, index=False, encoding='utf-8')
        df = pd.DataFrame(self.invalid_links, columns=["links"])
        df.to_csv(('invalid_links/' + self.file_name + '.csv'),header=True, index=False, encoding='utf-8')
    
    #save the robots from a url
    def save_robot(self):
        robot_url = self.url + "/robots.txt"
        req = requests.get(robot_url,headers=header)
        content = req.text
        save_file(content, "robots", (self.file_name + ".txt"))
    
    #getters and setters methods
    def get_url(self):
        return self.url
    def set_url(self, url):
        self.url = url
    def get_links_list(self):
        return self.links_list
    def set_links_list(self, links):
        self.links_list = links

In [262]:
for url in urls: 
    crawler = bfs_crawler(url)
    crawler.save_robot()
    crawler.get_links()
    crawler.save_as_csv()
    #print(crawler.robotParser.can_fetch("*", "https://www.shopfacil.com.br"))

Link Atual: http://www.magazineluiza.com.br/nossas-lojas
Link Atual: https://www.magazinevoce.com.br/?utm_campaign=header-site&utm_medium=referencia&utm_source=magazineluiza
Link Atual: https://especiais.magazineluiza.com.br/regulamentos/
Link Atual: http://especiais.magazineluiza.com.br/acessibilidade/
Link Atual: http://www.magazineluiza.com.br/acompanhamento
Link Atual: http://www.magazineluiza.com.br/central-de-atendimento
Link Atual: http://trocas.magazineluiza.com.br/
Link Atual: http://www.magazineluiza.com.br/seu-espaco
Link Atual: http://www.magazineluiza.com.br/favoritos
Link Atual: http://www.magazineluiza.com.br/cliente/logoff
Link Atual: http://www.magazineluiza.com.br/cliente/login
Link Atual: http://www.magazineluiza.com.br/#
Link Atual: https://www.magazineluiza.com.br/landingpage/?header=ofertasdodiaheader.png&unavailable=false&bob=true&menu=selecao-ofertasdodia&showcase=selecao-ofertasdodia
Link Atual: https://www.magazineluiza.com.br/celulares-e-telefones/l/te/
Link 

Link Atual: https://www.ricardoeletro.com.br/Loja/Cama-Mesa-e-Banho/2438
END
Link Atual: http://www.kabum.com.br/#
Link Atual: https://www.kabum.com.br/cgi-local/site/institucional/sobre.cgi
Link Atual: https://www.kabum.com.br/cgi-local/site/institucional/politicas.cgi
Link Atual: https://www.kabum.com.br/hardware
Link Atual: https://www.kabum.com.br/hardware/controladoras
Link Atual: https://www.kabum.com.br/hardware/rede-wireless-sem-fio
Link Atual: https://www.kabum.com.br/hardware/rede-wired-com-fio
Link Atual: https://www.kabum.com.br/hardware/ssd-2-5
Link Atual: https://www.kabum.com.br/hardware/placas-de-som
Link Atual: https://www.kabum.com.br/hardware/processadores
Link Atual: https://www.kabum.com.br/hardware/placas-mae
Link Atual: https://www.kabum.com.br/hardware/switch-kvm
Link Atual: https://www.kabum.com.br/hardware/placas-tv-edicao
Link Atual: https://www.kabum.com.br/hardware/modem
Link Atual: https://www.kabum.com.br/hardware/memoria-ram
Link Atual: https://www.kabum

Link Atual: http://www.avenida.com.br/feminino/colecao-feminina/cardigans-e-sueters?PS=12&O=OrderByReleaseDateDESC
Link Atual: http://www.avenida.com.br/feminino/colecao-feminina/casacos-e-jaquetas?PS=12&O=OrderByReleaseDateDESC
Link Atual: http://www.avenida.com.br/feminino/colecao-feminina/coletes-e-kimonos?PS=12&O=OrderByReleaseDateDESC
Link Atual: http://www.avenida.com.br/feminino/colecao-feminina/cropped?PS=12&O=OrderByReleaseDateDESC
Link Atual: http://www.avenida.com.br/feminino/colecao-feminina/macacoes-e-jardineiras?PS=12&O=OrderByReleaseDateDESC
END
